## 101 - Training and Evaluating Classifiers with `mmlspark`

In this example, we try to predict incomes from the *Adult Census* dataset.

First, we import the packages (use `help(mmlspark)` to view contents),

In [1]:
import numpy as np
import pandas as pd

StatementMeta(SamplePool, 28, 1, Finished, Available)

Now let's read the data and split it to train and test sets:

In [2]:
data = spark.read.parquet("wasbs://publicwasb@mmlspark.blob.core.windows.net/AdultCensusIncome.parquet")
data = data.select(["education", "marital-status", "hours-per-week", "income"])
train, test = data.randomSplit([0.75, 0.25], seed=123)
train.limit(10).toPandas()

StatementMeta(SamplePool, 28, 2, Finished, Available)

  education       marital-status  hours-per-week  income
0      10th             Divorced             1.0   <=50K
1      10th             Divorced            40.0   <=50K
2      10th             Divorced            40.0   <=50K
3      10th             Divorced            40.0   <=50K
4      10th             Divorced            40.0   <=50K
5      10th             Divorced            40.0   <=50K
6      10th   Married-civ-spouse            20.0   <=50K
7      10th   Married-civ-spouse            40.0   <=50K
8      10th   Married-civ-spouse            40.0   <=50K
9      10th   Married-civ-spouse            40.0   <=50K

`TrainClassifier` can be used to initialize and fit a model, it wraps SparkML classifiers.
You can use `help(mmlspark.TrainClassifier)` to view the different parameters.

Note that it implicitly converts the data into the format expected by the algorithm: tokenize
and hash strings, one-hot encodes categorical variables, assembles the features into a vector
and so on.  The parameter `numFeatures` controls the number of hashed features.

In [3]:
from mmlspark.train import TrainClassifier
from pyspark.ml.classification import LogisticRegression
model = TrainClassifier(model=LogisticRegression(), labelCol="income", numFeatures=256).fit(train)

StatementMeta(SamplePool, 28, 3, Finished, Available)

After the model is trained, we score it against the test dataset and view metrics.

In [4]:
from mmlspark.train import ComputeModelStatistics, TrainedClassifierModel
prediction = model.transform(test)
metrics = ComputeModelStatistics().transform(prediction)
metrics.limit(10).toPandas()

StatementMeta(SamplePool, 28, 4, Finished, Available)

  evaluation_type  ...       AUC
0  Classification  ...  0.865245

[1 rows x 6 columns]
/opt/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:2110: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: MatrixUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)

Finally, we save the model so it can be used in a scoring program.

In [5]:
model.write().overwrite().save("AdultCensus.mml") # save to abfss with simple data 

StatementMeta(SamplePool, 28, 5, Finished, Available)

In [7]:
spark.stop()

StatementMeta(SamplePool, 28, 7, Finished, Available)